Boundary Conditions
====

Usually, Dirichlet boundary conditions (essential bc) are built into the space: find $u \in H^1$ such that $u = u_D$ on $\Gamma_D$. Now, we want to pose the Dirichlet bc by an extra equation. For this, we start from the strong form

$$
-\Delta u = f,
$$

multiply the  equation by a test-function in $H^1$, integate by parts, and keep the boundary term:

$$
\int \nabla u \nabla v - \int_{\partial \Omega} \frac{\partial u}{\partial n} v = \int f v
$$

We introduce a new variable $\lambda$ for $-\frac{\partial u}{\partial n}$ on the Dirichlet boundary. Natural bc are treated as usual, we assume we only have homogeneous Neumann bc for ease of notation:

$$
\int \nabla u \nabla v + \int_{\Gamma_D} \lambda v = \int fv
$$

The Dirichlet bc $u = u_D$ is now enforced also by a test function $\mu$ living on the Dirichlet boundary. Thus, the whole equation is now:

Find $u \in V := H^1(\Omega)$ and $\lambda \in Q := H^{-1/2}(\Gamma_D)$ such that

$$
\begin{array}{ccccll}
\int_\Omega \nabla u \nabla v & + & \int_{\Gamma_D} \lambda v & = & \int_\Omega f v & \forall \, v \in V \\
\int_{\Gamma_D} \mu u & & & = & \int_{\Gamma_D} \mu u_D & \forall \, \mu \in Q
\end{array}
$$

Why do we have the space $H^{-1/2}(\Gamma_D)$ ?
Functions from $H^1(\Omega)$ have boundary values (the so called trace) exactly in the space $H^{1/2}(\Gamma_D)$. We can pair these functions with elements from its dual space, called $H^{-1/2}(\Gamma_D)$. To be precise, the integral is a convenient notation for the duality pairing:

$$
\int_{\Gamma_D} u \mu  \quad \text{in sense of} \quad \left< u|_{\Gamma_D}, \mu \right>_{H^{1/2}(\Gamma_D) \times H^{-1/2}(\Gamma_D)}
$$

The space $H^{-1/2}$ is weaker (i.e. larger) than $L_2$. Thus, we can use discontinuous $L_2$ finite elements for its discretization.

In [ ]:
import netgen.gui
from netgen.csg import unit_cube
from netgen.geom2d import unit_square
from ngsolve import * 
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
# mesh = Mesh(unit_cube.GenerateMesh(maxh=0.2))
Draw (mesh)

In [ ]:
V = H1(mesh, order=3, orderface=4)
Q = SurfaceL2(mesh, order=1)
# Q = H1(mesh, order=3, definedon="x", definedonbound=[1,2,3,4,5,6])
X = FESpace([V,Q])
print ("V.ndof =", V.ndof, "Q.ndof =", Q.ndof)

In [ ]:
u,lam = X.TrialFunction()
v,mu = X.TestFunction()
SetHeapSize(10000000)
a = BilinearForm(X)
a += grad(u)*grad(v)*dx + (u*mu+v*lam)*ds

f = LinearForm(X)
f += 10*x*v*dx + y*mu*ds

a.Assemble()
f.Assemble()

gfu = GridFunction(X)
gfu.vec.data = a.mat.Inverse(X.FreeDofs()) * f.vec

sol_u, sol_lam = gfu.components
Draw (sol_u, mesh, "u")
Draw (sol_lam, mesh, "lam")

Choosing the test-function $v = 1$ in the first equation 
$$
\int_\Omega \nabla u \nabla 1 + \int_{\Gamma_D} \lambda 1 = \int_\Omega f 1
$$
we observe that the total flux is exactly in balance with the total source

$$
\int_{\Gamma_D} \lambda = \int_{\Omega} f
$$

We compute the integral over the whole boundary as

In [ ]:
Integrate(sol_lam, mesh, BND)

In [ ]:
Integrate(sol_lam, mesh, BND, region_wise=True)